In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

### Airline Data Scraping

In [23]:
def extract_from_skytrax(url, airline, soup):
    rating, header,author, date, place, verification, content, collect=[],[],[],[],[],[],[],[]
    urls=[url]
    c=1
    while(soup!=None):
        articles = soup.find_all("article", attrs={'class':re.compile('comp comp_media-review-rated list-item media position-content review-[0-9]*')})
        for article in articles:
            if(article.find("div",class_='rating-10').find('span') is None):
                rating.append(None)
            else:
                rating.append(article.find("div",class_='rating-10').find('span').text)
            if(article.find("div",class_='body').find('h2') is None):
                header.append(None)
            else:
                header.append(article.find("div",class_='body').find('h2').text.strip("\""))
            if(article.find("div",class_='body').find('span',itemprop='author').find('span') is None):
                author.append(None)
            else:
                author.append(article.find("div",class_='body').find('span',itemprop='author').find('span').text)
            if(article.find("div",class_='body').find('time',itemprop='datePublished') is None):
                date.append(None)
            else:
                date.append(article.find("div",class_='body').find('time',itemprop='datePublished')['datetime'])
            if(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()) is None):
                place.append(None)
            else: 
                place.append(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()).group(1).strip('()'))
            content.append(article.find("div",class_='text_content').text.strip())
            tr= article.find("div",class_='body').find('table', class_='review-ratings').findAll(lambda tag: tag.name=='tr')
            table_dict={}
            for row in tr:
                key= row.find('td',attrs={'class':re.compile('review-rating-header.*')}).text
                if(row.find('td',class_='review-value')==None):
                    if(not [i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')]):
                        value=None
                    else:
                        value=max([i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')])
                else:
                    value= row.find('td',class_='review-value').text
                table_dict[key]=value
            collect.append(table_dict)
        next_article=soup.find('article', class_='comp comp_reviews-pagination querylist-pagination position-')
        c+=1
        next_url="https://www.airlinequality.com"+f"/airline-reviews/{airline}/page/"+str(c)+"/"
        urls.append(next_url)
        page= requests.get(next_url)
        soup= BeautifulSoup(page.content,"html.parser")
        if(c==367):
            break
        
    airline_review = pd.DataFrame({"rating":rating, 
                               "header":header, 
                               "author":author, 
                               "date":date,
                               "place":place,
                               "content":content, 
                               "collect":collect,
                               'airline': [airline]*len(collect)})
    return airline_review


In [27]:
airline_sets = ['singapore-airlines','qatar-airways',
                'emirates','japan-airlines','southwest-airlines','air-france'
                ,'korean-air','british-airways','asiana-airlines','airasia']

In [28]:
df_list = []
for airline in airline_sets:
    url = f"https://www.airlinequality.com/airline-reviews/{airline}"
    page= requests.get(url)
    soup= BeautifulSoup(page.content,"html.parser")
    df_list.append(extract_from_skytrax(url, airline, soup))

### Cleaning the data

In [94]:
def clean_data(airline_review):
    airline_review.dropna(inplace=True)
    airline_review.rating=airline_review.rating.astype(int)
    airline_review.date=pd.to_datetime(airline_review.date)
    new_airline_review=airline_review.merge(pd.json_normalize(airline_review.collect).rename(columns={
    'Aircraft':'aircraft',
    'Type Of Traveller':'traveller_type',
    'Seat Type':'seat_type',
    'Route':'route',
    'Date Flown':'date_flown',
    'Seat Comfort':'seat_comfort',
    'Cabin Staff Service':'cabin_staff_service', 
    'Food & Beverages':'food_beverages',
    'Ground Service':'ground_service',
    'Value For Money':'value_for_money',
    'Recommended':'recommended',
    'Wifi & Connectivity':'wifi_connectivity',
    'Inflight Entertainment':'entertainment'    
    }), left_index=True, right_index=True).drop(columns=['collect'])
    new_airline_review['trip_verified']=new_airline_review.content.str.split('|').apply(lambda x: x[0] if len(x)>=2 else None)
    new_airline_review.content = new_airline_review.content.str.split('|').apply(lambda x: x[1] if len(x)>=2 else x[0])
    return new_airline_review

In [115]:
final_data = clean_data(df_list[0])
for df in df_list[1:]:
    final_data = pd.concat([final_data, clean_data(df)], axis =0)

In [116]:
final_data.isnull().sum()

rating                     0
header                     0
author                     0
date                       0
place                      0
content                    0
airline                    0
traveller_type          3706
seat_type                 19
route                   3737
date_flown              3726
seat_comfort             728
cabin_staff_service      761
food_beverages          1956
entertainment           3886
ground_service          4044
wifi_connectivity      10973
value_for_money           19
recommended                0
aircraft                8514
trip_verified           5884
dtype: int64

In [117]:
final_data.head()

,rating,header,author,date,place,content,airline,traveller_type,seat_type,route,...,seat_comfort,cabin_staff_service,food_beverages,entertainment,ground_service,wifi_connectivity,value_for_money,recommended,aircraft,trip_verified
0,3,food were served last,Bobby Jayadevan,2024-04-12,India,From Kochi to Singapore and vice verse we t...,singapore-airlines,Couple Leisure,Economy Class,Kochi to Singapore,...,1,2,1,3,3,4,2,no,NaN,Not Verified
1,5,"seat spacing is less""",1 reviews,2024-04-07,Singapore,"The flight went from Terminal 1, the termin...",singapore-airlines,Business,Economy Class,Singapore to Sydney,...,1,5,4,5,2,5,1,no,Boeing 777-300,✅ Trip Verified
2,10,service was efficient and friendly,47 reviews,2024-03-31,Singapore,SQ894 is the afternoon flight to HKG operate...,singapore-airlines,Solo Leisure,Business Class,Singapore to Hong Kong,...,5,5,5,5,5,5,5,yes,A350-900,✅ Trip Verified
3,1,“Very disappointed”,M Ackermann,2024-03-31,Germany,Very disappointed with Singapore Airlines. Da...,singapore-airlines,Solo Leisure,Business Class,Munich to Singapore,...,1,3,2,NaN,1,NaN,1,no,A350,✅ Trip Verified
4,1,incompetent customer service,C Gorsen,2024-03-30,Switzerland,I was flying business class from BCN to MLE ...,singapore-airlines,Couple Leisure,Business Class,Singapore to Male,...,4,5,3,3,1,4,1,no,NaN,✅ Trip Verified


In [118]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15386 entries, 0 to 943
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               15386 non-null  int32         
 1   header               15386 non-null  object        
 2   author               15386 non-null  object        
 3   date                 15386 non-null  datetime64[ns]
 4   place                15386 non-null  object        
 5   content              15386 non-null  object        
 6   airline              15386 non-null  object        
 7   traveller_type       11680 non-null  object        
 8   seat_type            15367 non-null  object        
 9   route                11649 non-null  object        
 10  date_flown           11660 non-null  object        
 11  seat_comfort         14658 non-null  object        
 12  cabin_staff_service  14625 non-null  object        
 13  food_beverages       13430 non-nu

In [119]:
final_data.rating.mean()

5.658585727284544

In [120]:
final_data.drop(columns={'wifi_connectivity'}, inplace=True)

In [121]:
final_data.date_flown = pd.to_datetime(final_data.date_flown)

In [122]:
final_data.seat_comfort=final_data.seat_comfort.fillna(-1)
final_data.seat_comfort=final_data.seat_comfort.astype(int)
final_data.cabin_staff_service=final_data.cabin_staff_service.fillna(-1)
final_data.cabin_staff_service=final_data.cabin_staff_service.astype(int)
final_data.food_beverages=final_data.food_beverages.fillna(-1)
final_data.food_beverages=final_data.food_beverages.astype(int)
final_data.ground_service=final_data.ground_service.fillna(-1)
final_data.ground_service=final_data.ground_service.astype(int)
final_data.value_for_money=final_data.value_for_money.fillna(-1)
final_data.value_for_money=final_data.value_for_money.astype(int)
final_data.entertainment=final_data.entertainment.fillna(-1)
final_data.entertainment=final_data.entertainment.astype(int)

In [123]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15386 entries, 0 to 943
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               15386 non-null  int32         
 1   header               15386 non-null  object        
 2   author               15386 non-null  object        
 3   date                 15386 non-null  datetime64[ns]
 4   place                15386 non-null  object        
 5   content              15386 non-null  object        
 6   airline              15386 non-null  object        
 7   traveller_type       11680 non-null  object        
 8   seat_type            15367 non-null  object        
 9   route                11649 non-null  object        
 10  date_flown           11660 non-null  datetime64[ns]
 11  seat_comfort         15386 non-null  int32         
 12  cabin_staff_service  15386 non-null  int32         
 13  food_beverages       15386 non-nu

In [124]:
final_data.head()

,rating,header,author,date,place,content,airline,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,entertainment,ground_service,value_for_money,recommended,aircraft,trip_verified
0,3,food were served last,Bobby Jayadevan,2024-04-12,India,From Kochi to Singapore and vice verse we t...,singapore-airlines,Couple Leisure,Economy Class,Kochi to Singapore,2024-04-01,1,2,1,3,3,2,no,NaN,Not Verified
1,5,"seat spacing is less""",1 reviews,2024-04-07,Singapore,"The flight went from Terminal 1, the termin...",singapore-airlines,Business,Economy Class,Singapore to Sydney,2024-02-01,1,5,4,5,2,1,no,Boeing 777-300,✅ Trip Verified
2,10,service was efficient and friendly,47 reviews,2024-03-31,Singapore,SQ894 is the afternoon flight to HKG operate...,singapore-airlines,Solo Leisure,Business Class,Singapore to Hong Kong,2024-01-01,5,5,5,5,5,5,yes,A350-900,✅ Trip Verified
3,1,“Very disappointed”,M Ackermann,2024-03-31,Germany,Very disappointed with Singapore Airlines. Da...,singapore-airlines,Solo Leisure,Business Class,Munich to Singapore,2024-03-01,1,3,2,-1,1,1,no,A350,✅ Trip Verified
4,1,incompetent customer service,C Gorsen,2024-03-30,Switzerland,I was flying business class from BCN to MLE ...,singapore-airlines,Couple Leisure,Business Class,Singapore to Male,2024-03-01,4,5,3,3,1,1,no,NaN,✅ Trip Verified


In [125]:
final_data.trip_verified.unique()

array(['Not Verified ', '✅ Trip Verified ', None, '❎ Not Verified ',
       '✅ Verified Review ', '❎ Unverified ',
       'I was travelling with two kids from Calicut to Philadelphia via Doha. At the airport the for some reason, my ticket numbers were missing on the boarding pass. There was a confusion over my PNR and the with some baggage tags and the time is running out. I would like to give a special mention to the manager on duty Mr Diljith. He jumped in right away and handled the hurdles one Not Verified ',
       'NotVerified ', '❌ Not Verified ', 'cNot Verified ',
       'We travelled economy from Manchester to Toronto via Heathrow The flight from Manchester to Heathrow and return was very good. The seats had plenty of room and it was easy to fit laptop size briefcases under the seat in front and still have room to stretch your legs. The service from the cabin crew was excellent. The leg from Heathrow to Toronto was not as comfortable but it was acceptable. The seats appeared na

In [126]:
final_data.trip_verified.fillna("Trip Verified", inplace=True)

In [127]:
final_data[final_data.trip_verified.str.contains("We travelled economy from Manchester")]

,rating,header,author,date,place,content,airline,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,entertainment,ground_service,value_for_money,recommended,aircraft,trip_verified
3403,4,British Airways customer review,V Carrapiett,2014-11-03,United Kingdom,) I did not see the attendants down our aisle ...,british-airways,NaN,Economy Class,NaN,NaT,3,3,3,1,-1,3,no,NaN,We travelled economy from Manchester to Toront...


In [128]:
final_data[final_data.trip_verified.str.contains("I was travelling with two kids from Calicut")]

,rating,header,author,date,place,content,airline,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,entertainment,ground_service,value_for_money,recommended,aircraft,trip_verified
912,10,I really appreciate his help,Ajay Yenugula,2019-07-24,United States,after the other with calm and composed. I re...,qatar-airways,Family Leisure,Economy Class,Calicut to Philadelphia via Doha,2019-07-01,5,5,5,5,5,5,yes,NaN,I was travelling with two kids from Calicut to...


In [129]:
final_data.loc[3403,'content']=final_data.loc[3403,'trip_verified']+final_data.loc[3403,'content']
final_data.loc[912,'content']=final_data.loc[912,'trip_verified']+final_data.loc[912,'content']

In [132]:
final_data.drop(columns=['trip_verified'], inplace=True)

In [136]:
final_data[['rating','header','content','airline','recommended']].to_csv("full_airline_review.csv")